<a href="https://colab.research.google.com/github/avinashmane/colab/blob/master/BS4_runrepeat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
# Base parameters
baseUrl='https://www.amazon.in'
brands=['brooks','adidas','asics','new balance','saucony','sketchers','nike']
product='running shoes'
urlSuffix='&ref=nb_sb_noss_2&i=shoes&bbn=1983396031'
 
proxies = {
  'http': 'http://10.10.1.10:3128',
  'https': 'http://10.10.1.10:1080',
}
 
# Headers for request
AGENT='Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:86.0) Gecko/20100101 Firefox/86.0'
# AGENT='Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
HEADERS = ({'User-Agent': AGENT,
            'Accept-Language': 'en-US'})
 
 
from bs4 import BeautifulSoup
import requests
import re
#patterns
urlPattern=re.compile(r"/(.*)/dp/(.*)/")
numberPattern=re.compile(r"([,\d]+.?\d*)")
# cleanSe_Rating= lambda x : x.str.extract("(\d*\.\d*)", expand=False)
cleanSe_number=lambda x : x.str.extract("([,\d]+.?\d*)",expand=False).str.replace( ',', '', regex=True)
 
def getUserId(x):
  try:
    return x.split("/")[3].split(".")[2] # '/gp/profile/amzn1.account.AH7F4SSSW67XKBU5KP2MZG3HREDA/ref=cm_cr_arp_d_gw_btm?ie=UTF8'
  except:
    return ''
 
def get_number(x):
  if not x: return ''
  try : return float(      numberPattern.findall(x.replace("One","1")
                )[0].replace( ',', '')) # get first number
  except BaseException as e: 
    print ("get_number() error :",x,e);
    return '';
 
def getSoup(urlPath):
    print(f"Fetching {urlPath}")
    new_webpage = requests.get(urlPath, headers=HEADERS)
 
    return BeautifulSoup(new_webpage.content, "lxml")
 
# Function to extract Product Title
def get_title(soup):
     
    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
 
        # Inner NavigatableString Object
        title_value = title.string
 
        # Title as a string value
        title_string = title_value.strip()
 
        # # Printing types of values for efficient understanding
        # print(type(title))
        # print(type(title_value))
        # print(type(title_string))
        # print()
 
    except AttributeError:
        title_string = ""   
 
    return title_string
 
# Function to extract Product Price
def get_price(soup):
 
    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()
 
    except AttributeError:
 
        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()
 
        except:     
            price = ""  
 
    return price
 
# Function to extract Product Rating
def get_rating(soup):
 
    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
         
    except AttributeError:
         
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = "" 
 
    return rating
 
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
         
    except AttributeError:
        review_count = ""   
 
    return review_count
 
# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()
 
    except AttributeError:
        available = "Not Available"
 
    return available    
 
 # Function to extract Availability Status
def get_review_details(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()
    except AttributeError:
        available = "Not Available"
 
    return available

In [93]:
#### Product Links

# The webpage URL
URL = baseUrl+"/s?k="+product.replace(" ","+")+urlSuffix

# Soup Object containing all data
soup = getSoup(URL)

# Fetch links as List of Tag Objects
links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

# Store the links
links_list = []

# Loop for extracting links from Tag Objects
for link in links:
    links_list.append(link.get('href'))

len(links_list)

Fetching https://www.amazon.in/s?k=brooks+running+shoes&ref=nb_sb_noss_2


64

In [94]:
##### Product Master
prod_list=[]
# Loop for extracting product details from each link 
for link in links_list:

    name_id=urlPattern.findall(link)
    if len(name_id)==1 and\
       len(name_id[0])==2:

      new_soup=getSoup(baseUrl+link)

      # Function calls to display all necessary product information
      prod_list.append({"Id":name_id[0][1],
                      "Name":name_id[0][0],
                      "Title": get_title(new_soup),
                      "Url":link,
                      "Price": get_price(new_soup),
                      "Rating": get_rating(new_soup),
                      "Reviews": get_review_count(new_soup),
                      "Availability": get_availability(new_soup)})
    else:
      print(f"skipping {link}")

Fetching https://www.amazon.in/Brooks-Peacoat-Running-Shoes-1103481D453/dp/B08BWN1L8Y/ref=sr_1_1?dchild=1&keywords=brooks+running+shoes&qid=1616862124&sr=8-1
Fetching https://www.amazon.in/Brooks-Mens-Glycerin-Running-Shoe/dp/B082DPFRT4/ref=sr_1_2?dchild=1&keywords=brooks+running+shoes&qid=1616862124&sr=8-2
Fetching https://www.amazon.in/Brooks-Adrenaline-Blackened-Running-1103491D093/dp/B086T6D8D4/ref=sr_1_3?dchild=1&keywords=brooks+running+shoes&qid=1616862124&sr=8-3
Fetching https://www.amazon.in/Force10-Liberty-Brook-2-Running-5152002100410/dp/B07H7Q7WHM/ref=sr_1_4?dchild=1&keywords=brooks+running+shoes&qid=1616862124&sr=8-4
Fetching https://www.amazon.in/Brooks-Mens-Launch-Running-Shoe/dp/B0828BXRZ4/ref=sr_1_5?dchild=1&keywords=brooks+running+shoes&qid=1616862124&sr=8-5
Fetching https://www.amazon.in/Power-Silva-Running-Shoes-9-8396986/dp/B07NGSPP2Q/ref=sr_1_6?dchild=1&keywords=brooks+running+shoes&qid=1616862124&sr=8-6
Fetching https://www.amazon.in/Brooks-Mens-Ravenna-Running-Sh

In [95]:
import pandas as pd
import altair as alt
prod_df=pd.DataFrame(prod_list)
prod_df.head()

,Id,Name,Title,Url,Price,Rating,Reviews,Availability
0,B08BWN1L8Y,Brooks-Peacoat-Running-Shoes-1103481D453,Brooks Men's Race Running Shoe,/Brooks-Peacoat-Running-Shoes-1103481D453/dp/B...,"₹ 8,618.00 - ₹ 29,089.00",4.7 out of 5 stars,"4,481 ratings",Not Available
1,B082DPFRT4,Brooks-Mens-Glycerin-Running-Shoe,Brooks Mens Glycerin 18 Running Shoe,/Brooks-Mens-Glycerin-Running-Shoe/dp/B082DPFR...,"₹ 13,299.00",4.8 out of 5 stars,11 ratings,Not Available
2,B086T6D8D4,Brooks-Adrenaline-Blackened-Running-1103491D093,Brooks Men's Adrenaline GTS 21,/Brooks-Adrenaline-Blackened-Running-1103491D0...,"₹ 9,611.00 - ₹ 12,394.00",5.0 out of 5 stars,2 ratings,Not Available
3,B07H7Q7WHM,Force10-Liberty-Brook-2-Running-5152002100410,Force10 (By Liberty) Men's Brook-2 Running Shoes,/Force10-Liberty-Brook-2-Running-5152002100410...,"₹ 1,030.00 - ₹ 1,599.00",4.2 out of 5 stars,8 ratings,Not Available
4,B0828BXRZ4,Brooks-Mens-Launch-Running-Shoe,Brooks Mens Launch 7 Running Shoe,/Brooks-Mens-Launch-Running-Shoe/dp/B0828BXRZ4...,"₹ 7,199.00 - ₹ 8,099.00",4.5 out of 5 stars,"1,041 ratings",Not Available


In [96]:
# prod_df.sort_values('Rating',ascending=False)

prod_df.Price=cleanSe_number(prod_df.Price)
prod_df.Rating=cleanSe_number(prod_df.Rating)
prod_df.Reviews=cleanSe_number(prod_df.Reviews)

prod_df=prod_df.rename(columns = {0:'Name',1:'Id'}) #.set_index('Id')

In [97]:
alt.Chart(prod_df).mark_point().encode(
    alt.Y("Rating:Q",
          scale=alt.Scale(zero=False)
    ),
    alt.X("Price:Q",scale=alt.Scale(zero=False)),
    size="Reviews:Q",
    fill="Rating:Q",
    color="Availability",
    tooltip=['Title','Reviews:N','Rating:Q','Price:Q']
)

alt.Chart(...)

In [ ]:
def getTagText(self):
  # print(f"{self.name} getTagText() in")
  try:
    return self.getText().replace("\n","")
  except:
    return ''
    # print(f"{self.name} getTagText() failed")

def getReviewPage(url):
  
  reviews=getSoup(url
                  ).find_all(class_="review-text")
  # print(len(reviews))
  reviewArr=[]

  for reviewSpan in reviews:
    r=reviewSpan.find_parents(class_='review')[0] # get first parents for each

    id=r.get('id')
    user=getTagText(r.find("span",attrs={"class":"a-profile-name"}))
    
    userId=getUserId(r.find("a",class_="a-profile").get('href'))
    rating=get_number(getTagText(r.find("i",attrs={"class":"review-rating","data-hook":"review-star-rating"}).span))
    date=getTagText(r.find("span",attrs={"class":"review-date"}))
    helpful=get_number(getTagText(r.find("span",{"class":"cr-vote-text"})).split(" ")[0])
    text=getTagText(r.find('span',class_="review-text").span)

    obj={"id":id,"user":user,"userId":userId,"rating":rating,"date":date,"helpful":helpful,"text":text}

    reviewArr.append(obj)
    # print(obj)
  
  if len(reviews)==0:
    raise EOFError('No reviews')

  return reviewArr

In [ ]:
allProd_reviews=pd.DataFrame()

# Loop for extracting product details from each link 
for i,p in prod_df.iterrows():

  prod_reviews=[]
  if p.Id==p.Id: #valid product
    reviewUrl=f"{baseUrl}/{p.Name}/product-reviews/{p.Id}"
    pageNo=0
    while True: 
      pageNo+=1
      url=reviewUrl+'?pageNumber=' + str(pageNo)
      try:
        reviewArr = getReviewPage(url)
      except BaseException as e:# EOFError:   # print("breaking while",e)
        break
      prod_reviews += reviewArr
      # break # only for testing

    prod_reviews=pd.DataFrame(prod_reviews)
    prod_reviews['prodId']=p.Id

    #append to all reviews
    allProd_reviews=allProd_reviews.append(prod_reviews) if not allProd_reviews.empty else prod_reviews 

Fetching https://www.amazon.in/Walmeck-Headphones-Retractable-Hands-Free-Multi-Point/product-reviews/B07JG585X3?pageNumber=1
Fetching https://www.amazon.in/TEZINICX-Bluetooth-Cancellation-Headphone-Smartphone/product-reviews/B08YWPT7DN?pageNumber=1
Fetching https://www.amazon.in/TEZINICX-Bluetooth-Cancellation-Headphone-Smartphone/product-reviews/B08YWPT7DN?pageNumber=2
Fetching https://www.amazon.in/REDOCKS-Bluetooth-Cancellation-Headphone-Smartphone/product-reviews/B08YR5J293?pageNumber=1
Fetching https://www.amazon.in/REDOCKS-Bluetooth-Cancellation-Headphone-Smartphone/product-reviews/B08YR5J293?pageNumber=2
Fetching https://www.amazon.in/esuav-Bluetooth-Headphones-Hands-Free-Microphone/product-reviews/B08NYL9Z39?pageNumber=1
Fetching https://www.amazon.in/esuav-Bluetooth-Headphones-Hands-Free-Microphone/product-reviews/B08NYL9Z39?pageNumber=2
Fetching https://www.amazon.in/JVC-HA-EC20BT-Headphones-Bluetooth-Connectivity/product-reviews/B077GVC4CK?pageNumber=1
Fetching https://www.a

In [ ]:
# allProd_reviews=allProd_reviews if not allProd_reviews.empty else allProd_reviews.append(prod_reviews)
allProd_reviews.shape

(9507, 8)

In [ ]:
if False:
  url='https://www.amazon.in/HAVELLS-STEALTH-DECORATIVE-CEILING-RESISTANT/product-reviews/B079ZW2H1L?pageNumber=1'
  soup=BeautifulSoup(requests.get(url, headers=HEADERS).content, "lxml"
                  )
  rt=soup.find_all(class_='review')[0]
  for x in soup.find_all(attrs={"data-hook":True}):
    indent="                        "[:len(x.find_parents())]
    print(f"{indent}{x.name} | {getTagText(x)} | {x.get('class')}")

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a products file.
def saveFile(fileName,df):
  fileName=fileName.replace(" ","_").replace("+","_")
  df.to_csv(fileName)
  uploaded = drive.CreateFile({'title': fileName})
  uploaded.SetContentFile(f"{fileName}")
  uploaded.Upload()
  print(f"Uploaded {fileName} with ID {uploaded.get('id')}")

saveFile(f"{product.replace(' ','_')}_products.csv",prod_df)
saveFile(f"{product.replace(' ','_')}_reviews.csv",allProd_reviews)

Uploaded clip-on_bluetooth_headphone_products.csv with ID 1juikzILUFBAPJJs6CbG5sfedpTw8jGfj
Uploaded clip-on_bluetooth_headphone_reviews.csv with ID 1hPGgLs-DYKeiXnNcoZvy_PHbb12xYWGL


In [74]:
import json
from absl import logging
logging.set_verbosity(logging.INFO)
logging.info("test")


URL='https://api.runrepeat.com/suggest.php?q=running+shoes&version=2'
URL='https://api.runrepeat.com/get-documents?from=0&size=500&filter[]=433&filter[]=435&f_id=1&c_id=4&orderBy=relevance&q=running+shoes'
# URL='https://api.runrepeat.com/get-documents?from=0&size=300&filter[]=433&filter[]=435&f_id=1&c_id=4&orderBy=popularity&q=running+shoes'
try:
  resp=requests.get(URL)
except BaseException as e:
  print(e)

INFO:absl:test


In [80]:
import pandas as pd
df=pd.DataFrame(resp.json()['products'])
df['type']='Current'
df['type'][df['is_new']]='New'
df['type'][df['discontinued']]='Discontinued'
df['brand_name']=df.brand.apply(lambda x: x['name'])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [88]:
df[df.slug=='nike-metcon-6']

,expert_reviews_count,is_new,discontinued,users_score,score,score_text,total_votes,users_votes_count,product_id,msrp,score_color,name,categories,brand,slug,expert_score,views,shops_count,deals,available_sizes,default_color,msrp_formatted,offers_count,review_link,type,brand_name


In [82]:
import altair as alt
ttips=df.columns.values
alt.Chart(df).mark_point().encode(
    alt.Y("score",scale=alt.Scale(zero=False)),
    alt.X("total_votes",scale=alt.Scale(type='log')),
    # size='total_votes:Q',
    color='type',
    fill='brand_name:N',
    tooltip=['name','score','expert_score','users_score:Q','total_votes:Q','msrp_formatted:N','review_link']
)

alt.Chart(...)

In [52]:
df.loc[0].deals

[{'color': {'gender': 'w',
   'id': 799577,
   'image_sizes': {'lg': {'height': 279, 'width': 600},
    'md': {'height': 176, 'width': 380},
    'sm': {'height': 116, 'width': 250},
    'xs': {'height': 28, 'width': 60}},
   'image_url': 'https://cdn.runrepeat.com/i/brooks/32368/brooks-women-s-ghost-12-purple-134a-{SIZE}.jpg',
   'name': 'Brooks Ghost 12 - Black/Hollyhock/Pink (063)',
   'sku': '063',
   'slug': 'black'},
  'color_id': 799577,
  'discount_percentage': None,
  'gender': 'women',
  'is_any_color': False,
  'price': None,
  'price_formatted': '',
  'price_local': None,
  'width': 'wide'},
 {'color': {'gender': 'm',
   'id': 799578,
   'image_sizes': {'lg': {'height': 252, 'width': 600},
    'md': {'height': 160, 'width': 380},
    'sm': {'height': 105, 'width': 250},
    'xs': {'height': 25, 'width': 60}},
   'image_url': 'https://cdn.runrepeat.com/i/brooks/32368/brooks-mens-ghost-12-running-shoe-black-pearl-oyster-9db4-{SIZE}.jpg',
   'name': 'Brooks Ghost 12 - Black/Pea